In [5]:
import numpy as np

import os
from os.path import  join

import pickle

In [3]:
#clean_data_path_val = "C://input//neurolab//val//clean"
clean_data_path_val = "D://test_neurolab//val//val//clean"

clean_data_val = []
for folder in os.listdir(clean_data_path_val):
    for file in os.listdir(join(clean_data_path_val, folder)):
        data = np.load(join(clean_data_path_val, folder, file))
        clean_data_val.append(data)
        
#noisy_data_path_val = "C://input//neurolab//val//noisy"
noisy_data_path_val = "D://test_neurolab//val//val//noisy"
noisy_data_val = []
for folder in os.listdir(noisy_data_path_val):
    for file in os.listdir(join(noisy_data_path_val, folder)):
        data= np.load(join(noisy_data_path_val, folder, file))
        noisy_data_val.append(data)

In [6]:
mlp = pickle.load(open('mlp.sav', 'rb'))

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [39]:
ixgrid = np.ix_(range(0, 80), range(80))
n = noisy_data_val[0][ixgrid].flatten().reshape(1, -1)
n.shape

(1, 6400)

In [41]:
mlp.predict(n)

array([[nan, nan, nan, ..., nan, nan, nan]])

In [26]:
mlp

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=None, nesterovs_momentum=True, power_t=0.5,
       random_state=0, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [33]:
a[0][:100]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [ ]:
def predict_by_mel(mel, model,debug = False):
    if (debug): print("mel size", mel.shape)
    predictions = []
    for i in range(mel.shape[0]//80 + 1):
        if (80*(i+1)-1 < mel.shape[0]):
            if (debug): print("current size", 80*i, 80*(i+1)-1)
                
            ixgrid = np.ix_(range(80*i, 80*(i+1)), range(80))
            sample = mel[ixgrid]
            sample = sample.flatten()
            
            current_prediction = model.predict(sample.reshape(1, -1))
            
            predictions.append(current_prediction)
            
            if (debug): print(current_prediction)
        elif(80 - (80*(i+1)-1 - mel.shape[0]) > 30): 
            
            if (debug): print("current size", 80*i, 80*(i+1)-1)
            ixgrid = np.ix_(range(80*i, mel.shape[0]), range(80))
            if (debug): print(mel[ixgrid].shape)                
            pad_matrix = np.pad(mel[ixgrid], ((0, 80 - mel[ixgrid].shape[0]), (0, 0)), 'reflect')
            pad_matrix = pad_matrix.flatten()  
            if (debug): print(pad_matrix.shape) 
            current_prediction = model.predict(pad_matrix.reshape(1, -1))   
            predictions.append(current_prediction)
        else:
            if (debug): print("not enough real data left", 80 - (80*(i+1)-1 - mel.shape[0]))
    av_predictions = np.mean(predictions) 
    if (debug) : print(predictions)   
    if(av_predictions > 0.5):
        final_prediction = 1
    else:
        final_prediction = 0
         
    return final_prediction
ex_data = clean_data[1]
predict_by_mel(ex_data, logreg, debug = True)